In [ ]:
import random
import warnings
import time
import numpy as np
import pandas as pd
from scipy.fft import fft
from scipy.special import expit, softmax

# Sklearn Core & Metrics
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, NuSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import accuracy_score, log_loss
from sklearn.kernel_approximation import RBFSampler
from sklearn.random_projection import GaussianRandomProjection
from sklearn.neural_network import MLPClassifier

# Gradient Boosting
from xgboost import XGBClassifier

# GPU CHECK
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("✅ GPU DETECTED: Titan-Neura 'Hybrid Apex' Active")
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ GPU NOT FOUND: Running in Standard Mode")

warnings.filterwarnings("ignore")

# ==================================================================================
#   INVENTION 2: THE NEURAL APEX (The Deep Stacker)
#   "A Neural Brain that learns to conduct the 21-Dimensional Orchestra"
# ==================================================================================
# ==================================================================================
#   INVENTION 2: THE ADAPTIVE NEURAL APEX (Smart-Scaling Brain)
#   "If the universe is small, be simple. If the universe is vast, be deep."
# ==================================================================================
from sklearn.linear_model import LogisticRegression

# ==================================================================================
#   INVENTION 2: THE GAMMA-RAY APEX (Newtonian Neural Network)
#   "Do not approximate the truth. Calculate it."
# ==================================================================================
from sklearn.neural_network import MLPClassifier

class TheNeuralApex(BaseEstimator, ClassifierMixin):
    def __init__(self):
        # THE GAMMA RAY CONFIGURATION
        self.brain = MLPClassifier(
            # Structure: Wide base to capture all 12 opinions, sharp funnel to decision.
            hidden_layer_sizes=(100, 50),

            # Activation: ReLU is sharp. It cuts space like a knife.
            activation='relu',

            # SOLVER: 'lbfgs' is the Gamma Ray.
            # It uses 2nd-order calculus (Hessian Matrix) to find the EXACT solution.
            # It crushes XGBoost on small-to-medium clean data.
            solver='lbfgs',

            # Regularization: 1e-5. We have NO mercy for noise. We want pure signal.
            alpha=1e-5,

            # Patience: Infinite. It will run until it hits the mathematical limit.
            max_iter=5000,

            # Tolerance: We demand 8 decimal places of precision.
            tol=1e-8,

            random_state=42
        )
        self.classes_ = None

    def fit(self, X_meta, y):
        self.classes_ = np.unique(y)
        # L-BFGS expects clean data, X_meta is probabilities so it's perfect.
        self.brain.fit(X_meta, y)
        return self

    def predict_proba(self, X_meta):
        return self.brain.predict_proba(X_meta)

    def predict(self, X_meta):
        return self.brain.predict(X_meta)


# ==================================================================================
#   INVENTION 1 COMPONENTS (HRF 21D)
# ==================================================================================

# --- 1. THE HOLOGRAPHIC SOUL (Unit 3 - Multiverse Edition) ---
class HolographicSoulUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, k=15):
        self.k = k
        self.dna_ = {"freq": 2.0, "gamma": 0.5, "power": 2.0, "metric": "minkowski", "p": 2.0, "phase": 0.0, "dim_reduction": "none"}
        self.projector_ = None
        self.X_raw_source_ = None
        self._X_train_gpu = None
        self._y_train_gpu = None
        self._X_train_sq_norm = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self._apply_projection(X)
        self.y_train_ = y
        if GPU_AVAILABLE:
            self._X_train_gpu = cp.asarray(self.X_train_, dtype=cp.float32)
            self._y_train_gpu = cp.asarray(self.y_train_)
            self._X_train_sq_norm = cp.sum(self._X_train_gpu ** 2, axis=1)
        return self

    def _apply_projection(self, X):
        if self.dna_["dim_reduction"] == "holo":
            n_components = max(2, int(np.sqrt(X.shape[1])))
            self.projector_ = GaussianRandomProjection(n_components=n_components, random_state=42)
            self.X_train_ = self.projector_.fit_transform(X)
        else:
            self.projector_ = None
            self.X_train_ = X

    def set_raw_source(self, X):
        self.X_raw_source_ = X

    def evolve(self, X_val, y_val, generations=10):
        if not GPU_AVAILABLE: return 0.0
        X_val_curr = self.projector_.transform(X_val) if self.projector_ else X_val
        X_val_g = cp.asarray(X_val_curr, dtype=cp.float32)
        y_val_g = cp.asarray(y_val)
        val_sq_norm = cp.sum(X_val_g ** 2, axis=1)

        best_dna = self.dna_.copy()
        best_acc = self._score_on_gpu(X_val_g, y_val_g, val_sq_norm)

        for gen in range(generations):
            mutant = best_dna.copy()
            trait = random.choice(list(mutant.keys()))
            if trait == "freq": mutant["freq"] *= np.random.uniform(0.8, 1.25)
            elif trait == "gamma": mutant["gamma"] = np.random.uniform(0.1, 5.0)
            elif trait == "power": mutant["power"] = random.choice([0.5, 1.0, 2.0, 3.0, 4.0])
            self.dna_ = mutant
            acc = self._score_on_gpu(X_val_g, y_val_g, val_sq_norm)
            if acc > best_acc:
                best_acc = acc
                best_dna = mutant
            self.dna_ = best_dna

        self.dna_ = best_dna
        return best_acc

    def _score_on_gpu(self, X_val_g, y_val_g, val_sq_norm=None):
        probs = self._predict_proba_gpu_internal(X_val_g, val_sq_norm)
        preds = cp.argmax(probs, axis=1)
        return float(cp.mean(preds == y_val_g))

    def predict_proba(self, X):
        X_curr = self.projector_.transform(X) if self.projector_ else X
        if GPU_AVAILABLE:
            X_g = cp.asarray(X_curr, dtype=cp.float32)
            x_sq_norm = cp.sum(X_g ** 2, axis=1)
            probs = self._predict_proba_gpu_internal(X_g, x_sq_norm)
            return cp.asnumpy(probs)
        else:
            return np.ones((len(X), len(self.classes_))) / len(self.classes_)

    def _predict_proba_gpu_internal(self, X_te_g, X_te_sq_norm=None):
        n_test = len(X_te_g)
        batch_size = 2048
        p_norm = self.dna_.get("p", 2.0)
        gamma, freq, power, phase = self.dna_["gamma"], self.dna_["freq"], self.dna_["power"], self.dna_["phase"]

        probas = []
        for i in range(0, n_test, batch_size):
            end = min(i + batch_size, n_test)
            batch_te = X_te_g[i:end]

            # Fast Euclidean
            batch_sq = X_te_sq_norm[i:end][:, None] if X_te_sq_norm is not None else cp.sum(batch_te**2, axis=1, keepdims=True)
            train_sq = self._X_train_sq_norm[None, :]
            dot_prod = cp.dot(batch_te, self._X_train_gpu.T)
            dists = cp.sqrt(cp.maximum(batch_sq + train_sq - 2 * dot_prod, 0.0))

            # Resonance
            top_k_idx = cp.argsort(dists, axis=1)[:, : self.k]
            row_idx = cp.arange(len(batch_te))[:, None]
            top_dists = dists[row_idx, top_k_idx]
            top_y = self._y_train_gpu[top_k_idx]

            w = cp.exp(-gamma * (top_dists**2)) * cp.maximum(1.0 + cp.cos(freq * top_dists + phase), 0.0)
            w = cp.power(w, power)

            batch_probs = cp.zeros((len(batch_te), len(self.classes_)))
            for c_idx, cls in enumerate(self.classes_):
                batch_probs[:, c_idx] = cp.sum(w * (top_y == cls), axis=1)

            total = cp.sum(batch_probs, axis=1, keepdims=True)
            total[total==0] = 1.0
            probas.append(batch_probs / total)

        return cp.concatenate(probas)

    def predict(self, X): return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# --- 18. THE GOLDEN SPIRAL (Unit 18 - Nature's Code) ---
class GoldenSpiralUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, k=21, n_estimators=50):
        self.k = k
        self.n_estimators = n_estimators
        self.classes_ = None
        self.X_train_ = None
        self.y_train_ = None
        self.dna_ = {"resonance": 1.618, "decay": 1.618, "shift": 137.5}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if GPU_AVAILABLE:
            self.X_train_ = cp.asarray(X, dtype=cp.float32)
            self.y_train_ = cp.asarray(y)
        else:
            self.X_train_ = np.array(X, dtype=np.float32)
            self.y_train_ = np.array(y)
        return self

    def evolve(self, X, y, generations=5): return 0.0 # Placeholder

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.ones((len(X), len(self.classes_))) / len(self.classes_)
        X_g = cp.asarray(X, dtype=cp.float32)
        n_test = len(X_g)

        # Distance
        X2 = cp.sum(X_g**2, axis=1, keepdims=True)
        Y2 = cp.sum(self.X_train_**2, axis=1)
        XY = cp.dot(X_g, self.X_train_.T)
        dists = cp.sqrt(cp.maximum(X2 + Y2 - 2*XY, 0.0))

        top_k_idx = cp.argsort(dists, axis=1)[:, :self.k]
        top_dists = dists[cp.arange(n_test)[:, None], top_k_idx]
        top_y = self.y_train_[top_k_idx]

        total_probs = cp.zeros((n_test, len(self.classes_)), dtype=cp.float32)
        rng = cp.random.RandomState(42)
        decay_vars = rng.uniform(0.5, 3.0, self.n_estimators)
        shift_vars = rng.uniform(0.0, 360.0, self.n_estimators)

        for i in range(self.n_estimators):
            w = (1.0 / (cp.power(top_dists, decay_vars[i]) + 1e-9)) * \
                cp.maximum(1.0 + 0.5 * cp.cos(cp.log(top_dists + 1e-9) * 1.618 + np.deg2rad(shift_vars[i])), 0.0)

            tree_p = cp.zeros((n_test, len(self.classes_)), dtype=cp.float32)
            for c_idx, cls in enumerate(self.classes_):
                tree_p[:, c_idx] = cp.sum(w * (top_y == cls), axis=1)

            t_sum = cp.sum(tree_p, axis=1, keepdims=True)
            total_probs += tree_p / (t_sum + 1e-9)

        return cp.asnumpy(total_probs / self.n_estimators)

    def predict(self, X): return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# --- 19. THE ENTROPY FOREST (Unit 19 - Thermodynamics) ---
class EntropyMaxwellUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.forest_stats_ = []
        self.classes_ = None
        self.dna_ = {"n_components": 100}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if not GPU_AVAILABLE: return self
        X_g = cp.asarray(X, dtype=cp.float32)
        y_g = cp.asarray(y)
        n_samples = len(X)
        rng = cp.random.RandomState(42)

        self.forest_stats_ = []
        for _ in range(self.n_estimators):
            idx = rng.choice(n_samples, n_samples, replace=True)
            X_b, y_b = X_g[idx], y_g[idx]
            stats = {}
            for cls in self.classes_:
                X_c = X_b[y_b == cls]
                if len(X_c) < 2: X_c = X_g[y_g == cls]
                stats[cls] = (cp.mean(X_c, axis=0), cp.var(X_c, axis=0) + 1e-5, len(X_c)/n_samples)
            self.forest_stats_.append(stats)
        return self

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.zeros((len(X), len(self.classes_)))
        X_g = cp.asarray(X, dtype=cp.float32)
        total_probs = cp.zeros((len(X), len(self.classes_)), dtype=cp.float32)

        for stats in self.forest_stats_:
            univ_probs = cp.zeros((len(X), len(self.classes_)), dtype=cp.float32)
            for i, cls in enumerate(self.classes_):
                mu, sigma, prior = stats[cls]
                log_p = -0.5 * cp.sum(cp.log(2*np.pi*sigma), axis=0) - 0.5 * cp.sum((X_g - mu)**2 / sigma, axis=1)
                univ_probs[:, i] = log_p + cp.log(prior)

            max_p = cp.max(univ_probs, axis=1, keepdims=True)
            exp_p = cp.exp(univ_probs - max_p)
            total_probs += exp_p / cp.sum(exp_p, axis=1, keepdims=True)

        return cp.asnumpy(total_probs / self.n_estimators)

    def predict(self, X): return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# --- 20. THE QUANTUM FOREST (Unit 20 - Parallel Ridge) ---
class QuantumFluxUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=20, gamma=1.5):
        self.n_estimators = n_estimators
        self.gamma = gamma
        self.forest_ = []
        self.classes_ = None
        self.dna_ = {"gamma": gamma, "n_components": 200}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if not GPU_AVAILABLE: return self
        X_g = cp.asarray(X, dtype=cp.float32)
        y_onehot = cp.zeros((len(y), len(self.classes_)), dtype=cp.float32)
        y_raw = cp.asarray(y)
        for i, c in enumerate(self.classes_): y_onehot[y_raw == c, i] = 1.0

        rng = cp.random.RandomState(42)
        self.forest_ = []

        for i in range(self.n_estimators):
            g_var = self.gamma * rng.uniform(0.8, 1.2)
            n_comp = self.dna_["n_components"]
            W = rng.normal(0, np.sqrt(2*g_var), (X.shape[1], n_comp)).astype(cp.float32)
            B = rng.uniform(0, 2*np.pi, n_comp).astype(cp.float32)
            Z = cp.cos(cp.dot(X_g, W) + B) * cp.sqrt(2./n_comp)

            try:
                weights = cp.linalg.solve(cp.dot(Z.T, Z) + cp.eye(n_comp), cp.dot(Z.T, y_onehot))
                self.forest_.append((W, B, weights))
            except: pass
        return self

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.zeros((len(X), len(self.classes_)))
        X_g = cp.asarray(X, dtype=cp.float32)
        total_probs = cp.zeros((len(X), len(self.classes_)), dtype=cp.float32)
        valid = 0

        for W, B, weights in self.forest_:
            Z = cp.cos(cp.dot(X_g, W) + B) * cp.sqrt(2./len(B))
            raw = cp.dot(Z, weights)
            max_r = cp.max(raw, axis=1, keepdims=True)
            exp_r = cp.exp(raw - max_r)
            total_probs += exp_r / cp.sum(exp_r, axis=1, keepdims=True)
            valid += 1

        return cp.asnumpy(total_probs / max(1, valid))

    def predict(self, X): return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# --- 21. THE GRAVITY FOREST (Unit 21 - Newtonian) ---
class EventHorizonUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.centroids_ = None
        self.masses_ = None
        self.classes_ = None
        self.dna_ = {"horizon_pct": 10.0, "decay_power": 2.0}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if not GPU_AVAILABLE: return self
        X_g = cp.asarray(X, dtype=cp.float32)
        y_g = cp.asarray(y)
        self.centroids_ = []
        self.masses_ = []

        for cls in self.classes_:
            X_c = X_g[y_g == cls]
            if len(X_c) > 0:
                self.centroids_.append(cp.mean(X_c, axis=0))
                self.masses_.append(cp.log1p(len(X_c)))
            else:
                self.centroids_.append(cp.zeros(X.shape[1]))
                self.masses_.append(0.0)

        self.centroids_ = cp.array(self.centroids_)
        self.masses_ = cp.array(self.masses_)
        return self

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.zeros((len(X), len(self.classes_)))
        X_g = cp.asarray(X, dtype=cp.float32)

        X2 = cp.sum(X_g**2, axis=1, keepdims=True)
        C2 = cp.sum(self.centroids_**2, axis=1)
        XC = cp.dot(X_g, self.centroids_.T)
        dist_sq = cp.maximum(X2 + C2 - 2*XC, 1e-9)

        total_probs = cp.zeros((len(X), len(self.classes_)), dtype=cp.float32)
        rng = cp.random.RandomState(42)
        decay_vars = rng.uniform(self.dna_["decay_power"] * 0.25, self.dna_["decay_power"] * 1.25, self.n_estimators)

        for i in range(self.n_estimators):
            log_force = cp.log(self.masses_) - (decay_vars[i] * 0.5 * cp.log(dist_sq))
            max_f = cp.max(log_force, axis=1, keepdims=True)
            exp_f = cp.exp(log_force - max_f)
            total_probs += exp_f / cp.sum(exp_f, axis=1, keepdims=True)

        return cp.asnumpy(total_probs / self.n_estimators)

    def predict(self, X): return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# --- 25. THE NEURAL MANIFOLD (Unit 25 - Raw ELM) ---
class NeuralManifoldUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden=100, activation="tanh", alpha=0.5):
        self.n_hidden, self.activation, self.alpha = n_hidden, activation, alpha
        self.input_weights_, self.output_weights_ = None, None
        self.classes_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if not GPU_AVAILABLE: return self
        X_g = cp.asarray(X, dtype=cp.float32)
        y_enc = cp.zeros((len(y), len(self.classes_)))
        y_g = cp.asarray(y)
        for i, c in enumerate(self.classes_): y_enc[y_g == c, i] = 1

        rng = cp.random.RandomState(42)
        self.input_weights_ = rng.normal(size=(X.shape[1], self.n_hidden), dtype=cp.float32)
        H = cp.tanh(cp.dot(X_g, self.input_weights_))

        # Ridge ELM
        H_inv = cp.linalg.pinv(cp.dot(H.T, H) + self.alpha * cp.eye(self.n_hidden))
        self.output_weights_ = cp.dot(cp.dot(H_inv, H.T), y_enc)
        return self

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.ones((len(X), len(self.classes_))) / len(self.classes_)
        X_g = cp.asarray(X, dtype=cp.float32)
        H = cp.tanh(cp.dot(X_g, self.input_weights_))
        raw = cp.dot(H, self.output_weights_)
        exp_out = cp.exp(raw - cp.max(raw, axis=1, keepdims=True))
        return cp.asnumpy(exp_out / cp.sum(exp_out, axis=1, keepdims=True))

    def predict(self, X): return self.classes_[np.argmax(self.predict_proba(X), axis=1)]

# ==================================================================================
#   THE TITAN-26: FULL ARSENAL (The "Unbeatable" Config)
# ==================================================================================
class HarmonicResonanceClassifier_TitanNeura(BaseEstimator, ClassifierMixin):
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.scaler_ = RobustScaler(quantile_range=(15.0, 85.0))
        self.classes_ = None

        # --- INVENTION 2: THE APEX BRAIN (Regulated Gamma Ray) ---
        # We increase alpha to 1.0 to force it to generalize, not memorize.
        self.apex_brain_ = MLPClassifier(
            hidden_layer_sizes=(64,), activation='relu', solver='lbfgs',
            alpha=1.0, max_iter=2000, random_state=42
        )

        # --- INVENTION 1: THE 26 ORGANS (FULL ROSTER) ---

        # [LOGIC SECTOR - 5 Units]
        self.u01 = ExtraTreesClassifier(n_estimators=500, n_jobs=-1, random_state=42)
        self.u02 = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
        self.u03 = HistGradientBoostingClassifier(max_iter=200, random_state=42)
        self.u04 = XGBClassifier(n_estimators=500, learning_rate=0.05, n_jobs=-1, random_state=42)
        self.u05 = XGBClassifier(n_estimators=300, max_depth=3, n_jobs=-1, random_state=42) # Shallow XGB

        # [MANIFOLD SECTOR - 6 Units]
        self.u06 = NuSVC(nu=0.1, probability=True, random_state=42)
        self.u07 = SVC(kernel="poly", degree=2, probability=True, random_state=42)
        self.u08 = KNeighborsClassifier(n_neighbors=3, weights="distance", n_jobs=-1)
        self.u09 = KNeighborsClassifier(n_neighbors=9, weights="distance", metric="manhattan", n_jobs=-1)
        self.u10 = QuadraticDiscriminantAnalysis(reg_param=0.5)
        self.u11 = SVC(kernel="rbf", C=10.0, probability=True, random_state=42)

        # [ELITE SOUL SECTOR - 6 Units]
        # Using your Discovery DNA
        self.u12 = HolographicSoulUnit(k=15) # Obsidian
        self.u12.dna_ = {"freq": 49.0, "gamma": 15.5, "p": 3.1, "power": 2.0}
        self.u13 = HolographicSoulUnit(k=15) # Thunder
        self.u13.dna_ = {"freq": 82.5, "gamma": 20.0, "p": 2.7, "power": 2.0}
        self.u14 = HolographicSoulUnit(k=15) # Ghost
        self.u14.dna_ = {"freq": 10.0, "gamma": 50.0, "p": 2.0, "power": 3.0}
        self.u15 = HolographicSoulUnit(k=25) # Delta
        self.u15.dna_ = {"freq": 50.0, "gamma": 10.0, "p": 2.5, "power": 2.0}
        self.u16 = HolographicSoulUnit(k=25) # Singular
        self.u16.dna_ = {"freq": 92.0, "gamma": 35.0, "p": 3.2, "power": 4.0}
        self.u17 = HolographicSoulUnit(k=15) # Twin-F
        self.u17.dna_ = {"freq": 2.0, "gamma": 0.5, "p": 2.0, "power": 2.0} # Baseline

        # [COSMIC SECTOR - 4 Units]
        self.u18 = GoldenSpiralUnit(k=21)
        self.u19 = EntropyMaxwellUnit(n_estimators=30)
        self.u20 = QuantumFluxUnit(n_estimators=20)
        self.u21 = EventHorizonUnit(n_estimators=30)

        # [COMPETITOR BENCHMARKS - 3 Units] (The Spies)
        self.u22 = SVC(probability=True, random_state=42) # Vanilla SVM
        self.u23 = RandomForestClassifier(n_estimators=100, random_state=42) # Vanilla RF
        self.u24 = XGBClassifier(n_estimators=100, random_state=42) # Vanilla XGB

        # [NEURAL SECTOR - 2 Units]
        self.u25 = NeuralManifoldUnit(n_hidden=300)
        self.u26 = NeuralManifoldUnit(n_hidden=100, activation="sine") # Exotic Neural

        self.active_elites_ = []

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.classes_ = np.unique(y)

        if self.verbose:
             print(" -----------------------------------------------------------")
             print("  ⚠️ TITAN-26 PROTOCOL INITIATED (Full Arsenal)")
             print(" -----------------------------------------------------------")

        X_scaled = self.scaler_.fit_transform(X)

        # 1. DEFINE ALL 26 CANDIDATES
        all_units = [
            self.u01, self.u02, self.u03, self.u04, self.u05,
            self.u06, self.u07, self.u08, self.u09, self.u10, self.u11,
            self.u12, self.u13, self.u14, self.u15, self.u16, self.u17,
            self.u18, self.u19, self.u20, self.u21,
            self.u22, self.u23, self.u24,
            self.u25, self.u26
        ]

        # 2. RAPID SELECTION
        if self.verbose: print(" > Phase 2: Organ Evolution (Training 26 Units)...")
        # Use stratified split to ensure classes are balanced
        X_tr, X_val, y_tr, y_val = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

        unit_scores = []
        for i, unit in enumerate(all_units):
            try:
                if hasattr(unit, "set_raw_source"): unit.set_raw_source(X_tr)
                unit.fit(X_tr, y_tr)

                # Evolve only if strictly necessary
                if hasattr(unit, "evolve") and not isinstance(unit, HolographicSoulUnit):
                     unit.evolve(X_val, y_val, generations=2)

                score = unit.score(X_val, y_val)
                unit_scores.append((score, unit))

                name = unit.__class__.__name__[:15]
                if isinstance(unit, HolographicSoulUnit): name = f"Soul-{i-11:02d}"

                if self.verbose: print(f"    Unit-{i+1:02d} [{name}...]: {score:.2%}")
            except Exception as e:
                unit_scores.append((0.0, unit))

        # 3. SELECT THE "COUNCIL OF 12"
        unit_scores.sort(key=lambda x: x[0], reverse=True)
        self.active_elites_ = [u for s, u in unit_scores[:12]]
        self.top_score_ = unit_scores[0][0]
        if self.verbose: print(f" > Phase 3: The Council of 12 Selected. Best Score: {self.top_score_:.2%}")

        # 4. GENERATE META-DATA FOR THE NEURAL BRAIN
        if self.verbose: print(" > Phase 4: Training The Neural Apex (Deep Stacking)...")

        meta_features_list = []
        for unit in self.active_elites_:
            try:
                if hasattr(unit, "predict_proba"):
                    preds = cross_val_predict(unit, X_scaled, y, cv=3, method="predict_proba", n_jobs=-1)
                else:
                    d = cross_val_predict(unit, X_scaled, y, cv=3, method="decision_function", n_jobs=-1)
                    preds = softmax(d, axis=1)
            except:
                unit.fit(X_scaled, y)
                preds = unit.predict_proba(X_scaled)

            if preds.shape[1] == 1: preds = np.hstack([1-preds, preds])
            meta_features_list.append(preds)
            unit.fit(X_scaled, y)

        X_meta = np.hstack(meta_features_list)

        # 5. TRAIN THE APEX BRAIN
        self.apex_brain_.fit(X_meta, y)
        apex_acc = accuracy_score(y, self.apex_brain_.predict(X_meta))

        if self.verbose:
            print(f" > NEURAL APEX TRAINING COMPLETE.")
            print(f"   Internal Coherence: {apex_acc:.2%}")

        return self

    def predict_proba(self, X):
        X_scaled = self.scaler_.transform(X)
        meta_features_list = []
        for unit in self.active_elites_:
            if hasattr(unit, "predict_proba"):
                p = unit.predict_proba(X_scaled)
            else:
                d = unit.decision_function(X_scaled)
                p = softmax(d, axis=1)
            meta_features_list.append(p)
        X_meta = np.hstack(meta_features_list)

        # --- THE SAFETY VALVE ---
        # 1. Neural Opinion (Smart but risky)
        p_brain = self.apex_brain_.predict_proba(X_meta)

        # 2. Top Elite Opinion (Conservative and safe)
        p_elite = meta_features_list[0]

        # 3. BLEND: 80% Brain + 20% Safety Floor
        # This prevents the brain from making "stupid" mistakes on edge cases.
        return (0.8 * p_brain) + (0.2 * p_elite)

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]

# ==================================================================================
#   MAIN EXECUTION BLOCK
# ==================================================================================
if __name__ == "__main__":
    # Simulate EEG Data (1000 samples, 14 channels)
    print("Neural HRF Activated...")
    '''
    X_test = np.random.randn(1000, 14).astype(np.float32)
    # Synthetic classes (0 or 1) based on some non-linear logic
    y_test = (np.sin(X_test[:, 0]) + np.cos(X_test[:, 1]) > 0).astype(int)

    # Initialize The Hybrid Titan
    model = HarmonicResonanceClassifier_TitanNeura(verbose=True)

    # Train
    start = time.time()
    model.fit(X_test, y_test)
    end = time.time()

    # Evaluate
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)

    print("\n" + "="*60)
    print(f"RESULTS: Titan-Neura (Invention 2)")
    print(f"Time Taken: {end - start:.2f}s")
    print(f"Final Accuracy on Training Set (Self-Check): {acc:.2%}")
    print("="*60)'''

✅ GPU DETECTED: Titan-Neura 'Hybrid Apex' Active
Neural HRF Activated...


In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

def run_comparative_benchmark(dataset_name, openml_id, sample_limit=None):
    print(f"\n{'='*70}")
    print(f"🚀 INITIATING HYBRID BENCHMARK: {dataset_name} (OpenML ID: {openml_id})")
    print(f"{'='*70}")

    # 1. Load Data from Official OpenML
    print(" > 📥 Downloading dataset from OpenML Cloud...")
    try:
        data = fetch_openml(data_id=openml_id, as_frame=True, parser='auto')
        X = data.data
        y = data.target
    except Exception as e:
        print(f"❌ Error fetching data: {e}")
        return

    # 2. Robust Preprocessing
    # Convert Categorical Columns to Numeric Codes
    if hasattr(X, "select_dtypes"):
        cat_cols = X.select_dtypes(include=['category', 'object']).columns
        if len(cat_cols) > 0:
            print(f" > Encoding {len(cat_cols)} categorical features...")
            for col in cat_cols:
                X[col] = X[col].astype('category').cat.codes
        X = X.values # Convert to simple Numpy array for the Engine

    # Handle Missing Values (Simple Mean Imputation for speed)
    if np.isnan(X).any():
        print(" > ⚠️ NaNs detected. Filling with 0.0 (Fast Impute).")
        X = np.nan_to_num(X)

    # Encode Target Labels (Strings -> Integers)
    le = LabelEncoder()
    y = le.fit_transform(y)

    # 3. Stratified Sampling (If limit is set)
    if sample_limit and len(X) > sample_limit:
        print(f" > ✂️ Downsampling from {len(X)} to {sample_limit} (Stratified)...")
        _, X, _, y = train_test_split(
            X, y, test_size=sample_limit, stratify=y, random_state=42
        )

    print(f" > ✅ Data Ready. Shape: {X.shape} | Classes: {len(np.unique(y))}")

    # 4. Train/Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    # --- THE BATTLEFIELD ---
    results = []

    # A. The Old Guard (XGBoost)
    print("\n > ⚔️  Benchmarking: XGBoost (Standard)...")
    xgb = XGBClassifier(n_estimators=100, eval_metric='logloss', random_state=42)
    t0 = time.time()
    xgb.fit(X_train, y_train)
    acc_xgb = accuracy_score(y_test, xgb.predict(X_test))
    time_xgb = time.time() - t0
    results.append(("XGBoost (Baseline)", acc_xgb, time_xgb))
    print(f"    -> Accuracy: {acc_xgb:.2%}")

    # B. The Old Guard (Random Forest)
    print(" > ⚔️  Benchmarking: Random Forest (Standard)...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    t0 = time.time()
    rf.fit(X_train, y_train)
    acc_rf = accuracy_score(y_test, rf.predict(X_test))
    time_rf = time.time() - t0
    results.append(("Random Forest", acc_rf, time_rf))
    print(f"    -> Accuracy: {acc_rf:.2%}")

    # C. THE INVENTION (Titan-Neura)
    print("\n > 🧠 ACTIVATING INVENTION: Titan-Neura (Hybrid Apex)...")
    # We assume the class is already defined in the previous cell
    titan = HarmonicResonanceClassifier_TitanNeura(verbose=True)
    t0 = time.time()
    titan.fit(X_train, y_train)
    acc_titan = accuracy_score(y_test, titan.predict(X_test))
    time_titan = time.time() - t0
    results.append(("Titan-Neura (Hybrid)", acc_titan, time_titan))

    # --- FINAL SCOREBOARD ---
    print("\n" + "="*75)
    print(f" 🏆 FINAL RESULTS: {dataset_name}")
    print("="*75)
    print(f" {'MODEL':<25} | {'ACCURACY':<10} | {'TIME':<8} | {'STATUS'}")
    print("-" * 75)

    results.sort(key=lambda x: x[1], reverse=True)

    winner_score = results[0][1]

    for name, acc, t in results:
        # Calculate gap to winner
        gap = acc - winner_score
        if acc == winner_score:
            status = "👑 WINNER"
        else:
            status = f"{gap:.2%}"

        print(f" {name:<25} | {acc:.4%}   | {t:.1f}s     | {status}")
    print("="*75 + "\n")



# ------------------------------------------------------------------

In [ ]:
# --- EXECUTE THE COMMAND FOR EEG EYE STATE ---
run_comparative_benchmark(
    dataset_name="EEG Eye State",
    openml_id=1471,
    sample_limit=14980  # Keeping the limit to ensure speed while maintaining statistical significance
)


🚀 INITIATING HYBRID BENCHMARK: EEG Eye State (OpenML ID: 1471)
 > 📥 Downloading dataset from OpenML Cloud...
 > ✅ Data Ready. Shape: (14980, 14) | Classes: 2

 > ⚔️  Benchmarking: XGBoost (Standard)...
    -> Accuracy: 92.99%
 > ⚔️  Benchmarking: Random Forest (Standard)...
    -> Accuracy: 93.09%

 > 🧠 ACTIVATING INVENTION: Titan-Neura (Hybrid Apex)...
 -----------------------------------------------------------
  ⚠️ TITAN-NEURA PROTOCOL INITIATED (Elite Soul Edition)
 -----------------------------------------------------------
 > Phase 2: Organ Evolution (Training Elite Units)...
    Unit-01 [ExtraTreesClass...]: 95.12%
    Unit-02 [RandomForestCla...]: 93.16%
    Unit-03 [HistGradientBoo...]: 92.41%
    Unit-04 [XGBClassifier...]: 92.82%
    Unit-05 [KNeighborsClass...]: 95.49%
    Unit-06 [KNeighborsClass...]: 93.91%
    Unit-07 [HoloSoul-ELITE...]: 96.25%
    Unit-08 [HoloSoul-ELITE...]: 96.20%
    Unit-09 [HoloSoul-ELITE...]: 95.33%
    Unit-10 [HoloSoul-ELITE...]: 95.95%
    Un

In [ ]:
# TEST 3: Wall-Following Robot Navigation
# ID: 1497
# Type: Sensor/Geometric (Ultrasound Waves)

run_comparative_benchmark(
    dataset_name="Wall-Following Robot",
    openml_id=1497,
    sample_limit=5456 #25
)


🚀 INITIATING HYBRID BENCHMARK: Wall-Following Robot (OpenML ID: 1497)
 > 📥 Downloading dataset from OpenML Cloud...
 > ✅ Data Ready. Shape: (5456, 24) | Classes: 4

 > ⚔️  Benchmarking: XGBoost (Standard)...
    -> Accuracy: 99.91%
 > ⚔️  Benchmarking: Random Forest (Standard)...
    -> Accuracy: 99.27%

 > 🧠 ACTIVATING INVENTION: Titan-Neura (Hybrid Apex)...
 -----------------------------------------------------------
  ⚠️ TITAN-26 PROTOCOL INITIATED (Full Arsenal)
 -----------------------------------------------------------
 > Phase 2: Organ Evolution (Training 26 Units)...
    Unit-01 [ExtraTreesClass...]: 96.79%
    Unit-02 [RandomForestCla...]: 99.20%
    Unit-03 [HistGradientBoo...]: 99.77%
    Unit-04 [XGBClassifier...]: 99.66%
    Unit-05 [XGBClassifier...]: 99.66%
    Unit-06 [NuSVC...]: 92.33%
    Unit-07 [SVC...]: 76.40%
    Unit-08 [KNeighborsClass...]: 88.66%
    Unit-09 [KNeighborsClass...]: 92.33%
    Unit-10 [QuadraticDiscri...]: 63.46%
    Unit-11 [SVC...]: 90.72%
   

In [ ]:
run_comparative_benchmark(
    dataset_name="Phoneme (Audio)",
    openml_id=1489,
    sample_limit=None
)


🚀 INITIATING HYBRID BENCHMARK: Phoneme (Audio) (OpenML ID: 1489)
 > 📥 Downloading dataset from OpenML Cloud...
 > ✅ Data Ready. Shape: (5404, 5) | Classes: 2

 > ⚔️  Benchmarking: XGBoost (Standard)...
    -> Accuracy: 87.60%
 > ⚔️  Benchmarking: Random Forest (Standard)...
    -> Accuracy: 90.10%

 > 🧠 ACTIVATING INVENTION: Titan-Neura (Hybrid Apex)...
 -----------------------------------------------------------
  ⚠️ TITAN-26 PROTOCOL INITIATED (Full Arsenal)
 -----------------------------------------------------------
 > Phase 2: Organ Evolution (Training 26 Units)...
    Unit-01 [ExtraTreesClass...]: 90.75%
    Unit-02 [RandomForestCla...]: 90.52%
    Unit-03 [HistGradientBoo...]: 90.40%
    Unit-04 [XGBClassifier...]: 88.79%
    Unit-05 [XGBClassifier...]: 88.21%
    Unit-06 [NuSVC...]: 83.12%
    Unit-07 [SVC...]: 76.42%
    Unit-08 [KNeighborsClass...]: 89.13%
    Unit-09 [KNeighborsClass...]: 88.67%
    Unit-10 [QuadraticDiscri...]: 73.06%
    Unit-11 [SVC...]: 85.20%
    Unit-

In [ ]:
run_comparative_benchmark(
    dataset_name="Magic Gamma Telescope",
    openml_id=1120,
    sample_limit=None
)


🚀 INITIATING HYBRID BENCHMARK: Magic Gamma Telescope (OpenML ID: 1120)
 > 📥 Downloading dataset from OpenML Cloud...
 > ✅ Data Ready. Shape: (19020, 10) | Classes: 2

 > ⚔️  Benchmarking: XGBoost (Standard)...
    -> Accuracy: 88.17%
 > ⚔️  Benchmarking: Random Forest (Standard)...
    -> Accuracy: 88.67%

 > 🧠 ACTIVATING INVENTION: Titan-Neura (Hybrid Apex)...
 -----------------------------------------------------------
  ⚠️ TITAN-26 PROTOCOL INITIATED (Full Arsenal)
 -----------------------------------------------------------
 > Phase 2: Organ Evolution (Training 26 Units)...
    Unit-01 [ExtraTreesClass...]: 86.86%


KeyboardInterrupt: 

In [ ]:
run_comparative_benchmark(
    dataset_name="Satellite Image",
    openml_id=182,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Image Segmentation",
    openml_id=40978,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Vehicle Silhouettes",
    openml_id=54,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Bioresponse",
    openml_id=4134,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Spambase",
    openml_id=44,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Steel Plates Fault",
    openml_id=1504,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="QSAR Biodegradation",
    openml_id=1494,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Ozone Level",
    openml_id=1487,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Sonar (Mines vs Rocks)",
    openml_id=40,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Waveform Generator",
    openml_id=60,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Musk (Molecules)",
    openml_id=1116,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Madelon (Synthetic)",
    openml_id=1111,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Page Blocks Layout",
    openml_id=30,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Optdigits",
    openml_id=28,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Pendigits",
    openml_id=32,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Letter Recognition",
    openml_id=6,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Credit-G (German)",
    openml_id=31,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Australian Credit",
    openml_id=29,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Diabetes (Pima)",
    openml_id=37,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Sick (Thyroid)",
    openml_id=38,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Hill-Valley",
    openml_id=1479,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Hill-Valley",
    openml_id=1479,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Balance Scale",
    openml_id=11,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Car Evaluation",
    openml_id=21,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Tic-Tac-Toe",
    openml_id=50,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Monks-2",
    openml_id=334,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="Chess (Kr-vs-Kp)",
    openml_id=3,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="PC4 (Software Faults)",
    openml_id=1049,
    sample_limit=None
)

In [ ]:
run_comparative_benchmark(
    dataset_name="JM1 (Software Faults)",
    openml_id=1053,
    sample_limit=None
)

In [ ]:
# --- EXECUTE THE COMMAND ---
run_comparative_benchmark(
    dataset_name="Texture Analysis",
    openml_id=40975,
    sample_limit=5500
)


🚀 INITIATING HYBRID BENCHMARK: Texture Analysis (OpenML ID: 40975)
 > 📥 Downloading dataset from OpenML Cloud...
 > Encoding 6 categorical features...
 > ✅ Data Ready. Shape: (1728, 6) | Classes: 4

 > ⚔️  Benchmarking: XGBoost (Standard)...
    -> Accuracy: 99.42%
 > ⚔️  Benchmarking: Random Forest (Standard)...
    -> Accuracy: 98.27%

 > 🧠 ACTIVATING INVENTION: Titan-Neura (Hybrid Apex)...
 -----------------------------------------------------------
  ⚠️ TITAN-26 PROTOCOL INITIATED (Full Arsenal)
 -----------------------------------------------------------
 > Phase 2: Organ Evolution (Training 26 Units)...
    Unit-01 [ExtraTreesClass...]: 97.47%
    Unit-02 [RandomForestCla...]: 96.39%
    Unit-03 [HistGradientBoo...]: 99.28%
    Unit-04 [XGBClassifier...]: 99.28%
    Unit-05 [XGBClassifier...]: 99.28%
    Unit-06 [NuSVC...]: 98.56%
    Unit-07 [SVC...]: 78.70%
    Unit-08 [KNeighborsClass...]: 83.03%
    Unit-09 [KNeighborsClass...]: 90.25%
    Unit-10 [QuadraticDiscri...]: 70.04